In [ ]:
# 관련 모듈과 라이브러리 import하기

import torch.nn as nn
import torch 
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim 
import matplotlib.pyplot as plt

In [ ]:
# MNIST 데이터셋 다운로드 및 불러오기

# 데이터셋을 저장할 경로 넣을 변수
download_root = 'MNIST_data/'

# 학습용 데이터셋
dataset1 = datasets.MNIST(root=download_root, 
                          train = True, 
                          # 이미지를 PyTorch FloatTensor로 변환
                          # 왜냐면 딥러닝 모델은 텐서 형태의 데이터만 알아먹음!
                          # 픽셀값을 0~1 범위로 정규화하기!
                          transform = transforms.ToTensor(),
                          download=True)

# 테스트용 데이터셋
dataset2 = datasets.MNIST(root=download_root,
                            # 학습용이 아니므로 이번에는 False!
                            train = False, 
                            transform = transforms.ToTensor(),
                            download=True)






In [ ]:
# 모델 생성하기
